In [1]:
import plotly
import plotly.graph_objs as go
#import plotly.graph_objects as go
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from beakerx import *
#import ipypivot as pt
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox, HTML, ToggleButtons, interact, interact_manual
from bqplot import pyplot as plt
import bqplot

from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

In [2]:
df = pdr.get_data_yahoo("TSLA", start = "2018-07-01", end=pd.Timestamp('today'))
df2 = df

In [3]:
df2['SMA'] = df['Adj Close'].rolling(window=3).mean()

In [4]:
df2['Daily_Close'] = df2['Adj Close']

In [5]:
df2['Daily_Returns'] = df2['Daily_Close'].pct_change().round(4)
df2['Daily_LN_Returns'] = np.log(df2['Daily_Close'].pct_change()+1).round(4)
df2['Cum_Daily_Return'] = (1 + df2['Daily_Returns']).cumprod().round(4)
df2['Cum_Monthly_Return'] = df2['Daily_Close'].resample('BM').ffill().pct_change().round(4)

In [6]:
df2['Daily_Returns'].fillna(0, inplace = True)
df2['Daily_Close'].fillna(0, inplace = True)
df2['SMA'].fillna(0, inplace = True)
df2['Daily_LN_Returns'].fillna(0, inplace = True)
df2['Cum_Daily_Return'].fillna(0, inplace = True)
df2['Daily_Close'].fillna(0, inplace = True)

In [7]:
f = go.FigureWidget(
    data = [go.Scattergl(x = df2.index,
                         y = df2['Daily_Close'],
                         marker=dict(
                             color='#007dcc', size = 12,
                             line=dict(color = 'rgb(231, 99, 250)',
                                       width = 4)),
                         name='Daily Close',
                         yaxis='y1',
                         mode = 'markers'),
            go.Bar(x = df2.index,
                         y = df2['Daily_LN_Returns'],
                         name='Daily_Returns (log)',
                         yaxis='y2'),
            go.Scattergl(x = df2.index,
                         y = df2['SMA'].drop(df2.index[:3]),
                         marker=dict(
                             color='#1e4aa6', size = 12,
                             line=dict(color = 'rgb(231, 99, 250)',
                                       width = 4)),
                         name='3 Day Moving Average',
                         yaxis='y1',
                         mode = 'lines'),],
    layout = go.Layout(title = 'Favorite pick: Tesla - Year to Date Trend',
                       yaxis=dict(title='Closing Price in $.'),
                       yaxis2=go.layout.YAxis(anchor='x',
                                              overlaying='y',
                                              side='right',
                                              title="In Natural Logarithm"),
                       annotations=[go.layout.Annotation(
                                   x=df2['Daily_Close'].idxmin(),
                                   y=df2['Daily_Close'].min(),
                                   xref="x",
                                   yref="y",
                                   text="Buy:<br>"+str(df2['Daily_Close'].min().round(3)),
                                   showarrow=True,
                                   arrowhead=1,
                                   arrowsize=1,
                                   arrowcolor='black',
                                   ax=0,
                                   ay=-200)],
                       xaxis={"title":"Date",
                              'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 'step': 'month', 'stepmode': 'backward'},
                                                                 {'count': 6, 'label': '6M', 'step': 'month', 'stepmode': 'backward'},
                                                                 {'count': 12, 'label': '1Y', 'step': 'month', 'stepmode': 'backward'},
                                                                 {'step': 'all'}])}}
                      )
)

f.layout.update(
    autosize=False,
    width=900,
    height=500,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

scatter = f.data[0]
N = len(df2)
scatter.x = df2.index
scatter.y = df2['Daily_Close']
scatter.marker.opacity = 0.5

df2['Low'] = df2['Low'].round(4)
df2['High'] = df2['High'].round(4)
df2['Daily_Close'] = df2['Daily_Close'].round(4)


def update_axes(xaxis, yaxis):
    scatter = f.data[0]
    scatter.x = df2[xaxis]
    scatter.y = df2[yaxis]
    with f.batch_update():
        f.layout.xaxis.title = xaxis
        f.layout.yaxis.title = yaxis

t = go.FigureWidget([go.Table(
    header=dict(values=['Low (In $)','High (In $)','Daily Close (In $)','Daily_Returns (Natural Log)'],
                fill = dict(color='#386dea'),
                font=dict(color='#fcfcfc'),
                align = ['left'] * 5),
    cells=dict(values=[df2[col] for col in ['Low','High','Daily_Close','Daily_LN_Returns']],
                fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

def selection_fn(trace, points, selector):
    t.data[0].cells.values = [df2.iloc[points.point_inds][col] for col in ['Low', 'High','Daily_Close','Daily_LN_Returns']]

scatter.on_selection(selection_fn)

# Put everything together
VBox((HBox(),f))

    'data': [{'marker': {'color': '#007dcc',
                         'l…

In [8]:
t = go.FigureWidget([go.Table(
    header=dict(values=['Low (In $)','High (In $)','Daily Close (In $)','Daily_Returns (Natural Log)'],
                fill = dict(color='#386dea'),
                font=dict(color='#fcfcfc'),
                align = ['left'] * 5),
    cells=dict(values=[df2[col] for col in ['Low','High','Daily_Close','Daily_LN_Returns']],
                fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

t.layout.update(
    autosize=False,
    width=700,
    height=500,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

def selection_fn(trace, points, selector):
    t.data[0].cells.values = [df2.iloc[points.point_inds][col] for col in ['Low', 'High','Daily_Close','Daily_LN_Returns']]

scatter.on_selection(selection_fn)

# Put everything together
VBox((HBox(),t))

    'data': [{'cells': {'align': [left, left, left, left, left],
       …

In [9]:
h = go.FigureWidget(
    [go.Histogram(x=df2['Daily_Close'])])
    
h.layout.update(
    autosize=False,
    width=500,
    height=500,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
    
)

VBox((HBox(),h))

    'data': [{'type': 'histogram',
              'uid': '2ab46aa0-eecc-4…

In [10]:
y= df2['Daily_LN_Returns']
plt.figure()
plt.hist(y, colors=['OrangeRed'])
plt.show()